In [22]:
import pandas as pd
import numpy as np
import datetime as dt
import random as rd
import math
import quasardb
import quasardb.pandas as qdbpd

In [30]:
number_of_quotes=10
freq_quotes=5
date = dt.datetime (2017, 3, 14, 9, 30, 0, 0)

In [52]:
index = pd.date_range(date, periods=number_of_quotes, freq=str(freq_quotes)+'s')
jitter=np.random.randint(0, freq_quotes-1,number_of_quotes )
jitter_timedelta=pd.to_timedelta(jitter,unit='s')
index= index+jitter_timedelta


In [55]:

bids=np.random.randint(100, 110,number_of_quotes )
asks=bids+rd.randint(1,3)

In [57]:
quotes= pd.DataFrame (data={'bid_volume':np.random.randint(10, 110,number_of_quotes ),
                            'bid':bids,
                            'ask':asks,
                            'ask_volume':np.random.randint(10, 110,number_of_quotes )}
                      ,index=index)

In [58]:
quotes[:15]

,bid_volume,bid,ask,ask_volume
2017-03-14 09:30:02,21,108,110,43
2017-03-14 09:30:06,105,100,102,10
2017-03-14 09:30:10,15,105,107,32
2017-03-14 09:30:18,47,104,106,78
2017-03-14 09:30:20,62,101,103,84
2017-03-14 09:30:26,87,109,111,29
2017-03-14 09:30:30,54,105,107,12
2017-03-14 09:30:37,79,104,106,11
2017-03-14 09:30:41,77,109,111,101
2017-03-14 09:30:47,83,102,104,42


In [59]:
c = quasardb.Cluster("qdb://qdb-server:2836")

In [60]:
qdbpd.write_dataframe(quotes, c, "quotes",create=True)

In [61]:
query= c.query("select  last( bid ) from quotes in range ( 2017-03-14 ,2017-03-15 ) group by 2s ")
query_result = query.run()

In [62]:
tmp = dict()
for col in query_result.tables['quotes']:
    tmp[col.name] = col.data
    result_as_pandas = pd.DataFrame.from_dict(tmp, orient='columns').set_index('$timestamp')
result_as_pandas[:5]

,last(bid)
$timestamp,
2017-03-14 09:30:02,108
2017-03-14 09:30:06,100
2017-03-14 09:30:10,105
2017-03-14 09:30:18,104
2017-03-14 09:30:20,101
